In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv('earthquakedataset.csv',encoding='utf-8')
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,04/14/1970,13:46:30,-21.080,-174.311,Earthquake,25.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEMSUP796734,ISCGEMSUP,ISCGEM,ISCGEM,Automatic
1,04/14/1970,19:08:19,-33.308,19.486,Earthquake,10.0,NaN,NaN,5.7,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM796738,ISCGEM,ISCGEM,ISCGEM,Automatic
2,04/15/1970,13:14:25,15.219,122.624,Earthquake,25.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM796759,ISCGEM,ISCGEM,ISCGEM,Automatic
3,04/16/1970,01:55:57,34.579,141.629,Earthquake,25.0,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM796780,ISCGEM,ISCGEM,ISCGEM,Automatic
4,04/16/1970,05:33:21,59.702,-142.683,Earthquake,15.0,NaN,NaN,6.7,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM796787,ISCGEM,ISCGEM,ISCGEM,Automatic


In [3]:
df = df[['Date','Time','Latitude','Longitude','Depth','Magnitude']]
df.head(100)

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,04/14/1970,13:46:30,-21.080,-174.311,25.0,5.8
1,04/14/1970,19:08:19,-33.308,19.486,10.0,5.7
2,04/15/1970,13:14:25,15.219,122.624,25.0,6.2
3,04/16/1970,01:55:57,34.579,141.629,25.0,6.0
4,04/16/1970,05:33:21,59.702,-142.683,15.0,6.7
...,...,...,...,...,...,...
95,07/17/1970,20:04:49,-22.137,-174.647,35.0,6.3
96,07/18/1970,01:48:39,51.329,-178.508,35.0,6.1
97,07/19/1970,09:22:44,-4.056,152.156,20.0,6.2
98,07/23/1970,15:45:14,-15.834,-173.514,80.0,5.6


In [4]:
import datetime
import time

timestamp = []
for d, t in zip(df['Date'], df['Time']):
    try:
        timestring = datetime.datetime.strptime(d+' '+t, '%m/%d/%Y %H:%M:%S')
        timestamp.append(time.mktime(timestring.timetuple()))
    except ValueError:
        # print('ValueError')
        timestamp.append('ValueError')

In [5]:
df['Timestamp']= pd.DataFrame(timestamp)
df['Timestamp']=df.Timestamp != 'ValueError'

In [6]:
X = df[['Timestamp', 'Latitude', 'Longitude']]
y = df[['Magnitude', 'Depth']]
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

(17484, 3) (4371, 3) (17484, 2) (4371, 3)


In [7]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(neurons, activation, optimizer, loss):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

Using TensorFlow backend.


In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_model)


In [9]:
neurons = [16]

batch_size = [10, 20]
epochs = [10]
activation = ['relu','sigmoid']
optimizer = [ 'Adadelta','SGD']
loss = ['squared_hinge']

param_grid = dict(neurons=neurons, batch_size=batch_size, epochs=epochs, activation=activation, optimizer=optimizer, loss=loss)


In [10]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Epoch 1/10
17484/17484 [==============================] - 2s 139us/step - loss: 0.0044 - accuracy: 0.0198
Epoch 2/10
17484/17484 [==============================] - 2s 126us/step - loss: 0.0044 - accuracy: 0.0198
Epoch 3/10
17484/17484 [==============================] - 2s 124us/step - loss: 0.0044 - accuracy: 0.0198
Epoch 4/10
17484/17484 [==============================] - 2s 125us/step - loss: 0.0043 - accuracy: 0.0198
Epoch 5/10
17484/17484 [==============================] - 2s 125us/step - loss: 0.0043 - accuracy: 0.0198
Epoch 6/10
17484/17484 [==============================] - 2s 124us/step - loss: 0.0043 - accuracy: 0.0198
Epoch 7/10
17484/17484 [==============================] - 2s 127us/step - loss: 0.0043 - accuracy: 0.0198
Epoch 8/10
17484/17484 [==============================] - 2s 124us/step - loss: 0.0043 - accuracy: 0.0198
Epoch 9/10
17484/17484 [==============================] - 2s 125us/step - loss: 0.0043 - accuracy: 0.0198
Epoch 10/10
17484/17484 [=====================

In [11]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(3,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='Adadelta', loss='squared_hinge', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=10, epochs=10)

[test_loss, test_acc] = model.evaluate(X_test, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

Epoch 1/10
17484/17484 [==============================] - 5s 302us/step - loss: 0.0140 - accuracy: 0.4568
Epoch 2/10
17484/17484 [==============================] - 5s 274us/step - loss: 0.0043 - accuracy: 0.5410
Epoch 3/10
17484/17484 [==============================] - 5s 278us/step - loss: 0.0043 - accuracy: 0.5104
Epoch 4/10
17484/17484 [==============================] - 5s 272us/step - loss: 0.0042 - accuracy: 0.4447
Epoch 5/10
17484/17484 [==============================] - 5s 273us/step - loss: 0.0043 - accuracy: 0.4527
Epoch 6/10
17484/17484 [==============================] - 5s 277us/step - loss: 0.0043 - accuracy: 0.4733
Epoch 7/10
17484/17484 [==============================] - 5s 272us/step - loss: 0.0044 - accuracy: 0.6691
Epoch 8/10
17484/17484 [==============================] - 5s 272us/step - loss: 0.0042 - accuracy: 0.4166
Epoch 9/10
17484/17484 [==============================] - 5s 274us/step - loss: 0.0042 - accuracy: 0.3713
Epoch 10/10
4371/4371 [=======================